In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import cyrtranslit
from sklearn import preprocessing, model_selection, metrics, feature_selection, ensemble, linear_model, neural_network
from sklearn.pipeline import Pipeline
import lightgbm as lgb
color = sns.color_palette()
import pickle
%matplotlib inline

In [3]:
train = pd.read_pickle('../../train.pkl',compression='zip')
test = pd.read_pickle('../../test.pkl',compression='zip')

In [4]:
features = pd.read_pickle('../4.NLP Discretized Term Decomposition/allfeatures.pkl',compression='zip')

In [5]:
y = train.deal_probability
X_dev,X_val,y_dev,y_val = model_selection.train_test_split(features,y)

# Load RandomForest

In [6]:
rf = pickle.load(open('random_forest', 'rb'))

In [8]:
result = rf.score(X_val, y_val)
print(result)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.5s remaining:    0.0s


0.4278083853239413


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.9s finished


In [9]:
pred = rf.predict(X_val)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   17.8s finished


In [10]:
print('Less than zero:',sum(pred<0))
print('Over one:',sum(pred>1))

Less than zero: 0
Over one: 0


In [11]:
metrics.r2_score(y_val,pred)

0.4278083853239413

In [12]:
metrics.mean_squared_error(y_val,pred)

0.03870843793577741

In [13]:
metrics.mean_squared_error(y_val,pred)**0.5

0.196744600779227

- These scores are pretty good. RMSE is below the current best score on Kaggle. However, they are lower than the scores I got when I first created the model on the other notebook. That's a sign of overfitting. It became really good at predicting the train data on that particular random train/test split. Here, the splits were randomly re-assigned, and now the scores are lower.
- It's still not an issue since the final version of this model will be trained without actually splitting train into `X_dev` and `X_val`. That's because the competition already gives us our split sets in advance.

In [14]:
X_val= pd.read_pickle('X_val.pkl')
y_val= pd.read_pickle('y_val.pkl')
X_dev= pd.read_pickle('X_dev.pkl')
y_dev= pd.read_pickle('y_dev.pkl')

In [15]:
pred = rf.predict(X_val)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.7s finished


In [16]:
print('Less than zero:',sum(pred<0))
print('Over one:',sum(pred>1))

Less than zero: 0
Over one: 0


In [17]:
metrics.r2_score(y_val,pred)

0.8978347240344311

In [18]:
metrics.mean_squared_error(y_val,pred)

0.0068932069618381124

In [19]:
metrics.mean_squared_error(y_val,pred)**0.5

0.08302533927565796

- Those are the exact same score we got in the previous notebook, when we created the data. Confirmation of overfitting.

# Load LGBM

In [20]:
gb = pickle.load(open('light_gbm', 'rb'))

- Reshuffle splits.

In [23]:
y = train.deal_probability
X_dev,X_val,y_dev,y_val = model_selection.train_test_split(features,y)

In [24]:
pred = gb.predict(X_val)

In [25]:
print('Less than zero:',sum(pred<0))
print('Over one:',sum(pred>1))

Less than zero: 5290
Over one: 49


In [26]:
pred[pred>1] = 1
pred[pred<0] = 0

In [27]:
metrics.r2_score(y_val,pred)

0.34862806410952507

In [28]:
metrics.mean_squared_error(y_val,pred)

0.044222065772506224

In [29]:
metrics.mean_squared_error(y_val,pred)**0.5

0.21029043195663047

- This score is better than when we created it. Its still not as good as RandomForest. This score is also slightly superior to Kaggle's current first place, by a small margin... (`0.2200xxx`)

# Load MLPRegressor

In [30]:
mlp = pickle.load(open('mlpregressor', 'rb'))

In [31]:
pred = mlp.predict(X_val)

In [32]:
print('Less than zero:',sum(pred<0))
print('Over one:',sum(pred>1))

Less than zero: 3542
Over one: 0


In [33]:
pred[pred>1] = 1
pred[pred<0] = 0

In [34]:
metrics.r2_score(y_val,pred)

0.30550875907696307

In [35]:
metrics.mean_squared_error(y_val,pred)

0.04714946352814938

In [36]:
metrics.mean_squared_error(y_val,pred)**0.5

0.2171392721921794

- This score is the closest so far to the score at creation time. Virtually no change. It's yet closer to Kaggle's first place, but still superior.

# Load Keras

In [37]:
k = pickle.load(open('keras', 'rb'))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [38]:
pred = k.predict(X_val)

In [39]:
print('Less than zero:',sum(pred<0))
print('Over one:',sum(pred>1))

Less than zero: [0]
Over one: [80]


In [40]:
pred[pred>1] = 1
pred[pred<0] = 0

In [41]:
metrics.r2_score(y_val,pred)

0.2955757451050247

In [42]:
metrics.mean_squared_error(y_val,pred)

0.04782382232837276

In [43]:
metrics.mean_squared_error(y_val,pred)**0.5

0.21868658470142324

- Also no signs of overfitting. Lowest score in my comparison. Still better than Kaggle's first place.